# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [54]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from scikeras.wrappers import KerasRegressor

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [5]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data = data.dropna(subset=['total_pymnt'])


# Print the first five rows
data.head()


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [6]:
# Print the dimensions of data 
data.shape


(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [7]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [15]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer(strategy='mean')

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = ss_X.fit_transform(X_train_imputed)

# Transform test data
X_test_scaled = ss_X.transform(X_test_imputed)

#transform to DATAFRAME
X_train_scaled = pd.DataFrame(X_train_scaled, columns=cont_features)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [13]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat=X_train_cat.fillna('missing')
X_test_cat=X_test_cat.fillna('missing')




# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names_out(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe, columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe, columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [18]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]
X_train_all.head()

,loan_amnt,funded_amnt_inv,installment,annual_inc,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_missing
0,0.961300,0.487971,-0.013952,-0.326925,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.741417,-0.336399,0.812258,0.582909,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.487489,-0.374986,-0.316000,-0.143161,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.852220,-0.753845,-1.043301,-0.064069,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,3.227354,3.483765,4.076415,0.331394,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Standardize the target DataFrames (`y_train` and `y_test`) 

In [19]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [20]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [21]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [22]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)   

Epoch 1/150


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7065 - mse: 0.7065 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2650 - mse: 0.2650
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2424 - mse: 0.2424
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.2317 - mse: 0.2317
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - loss: 0.2307 - mse: 0.2307
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2185 - mse: 0.2185
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2213 - mse: 0.2213
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2222 - mse: 0.2222
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2134 - mse: 0.2134
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2035 - mse: 0.2035
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2068 - mse: 0.2068
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2122 - mse: 0.2122
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - loss: 0.7741 - mse: 0.7741
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2823 - mse: 0.2823
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2638 - mse: 0.2638
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2536 - mse: 0.2536
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2404 - mse: 0.2404
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2319 - mse: 0.2319
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2396 - mse: 0.2396
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - loss: 0.2281 - mse: 0.2281
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2240 - mse: 0.2240
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2238 - mse: 0.2238
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - loss: 0.2217 - mse: 0.2217
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2160 - mse: 0.2160
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 

...

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.5939 - mse: 0.5939
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3497 - mse: 0.3497
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2944 - mse: 0.2944
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2632 - mse: 0.2632
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2474 - mse: 0.2474
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2459 - mse: 0.2459
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - loss: 0.2409 - mse: 0.2409
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2249 - mse: 0.2249
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - loss: 0.2257 - mse: 0.2257
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2235 - mse: 0.2235
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2059 - mse: 0.2059
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2240 - mse: 0.2240
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4852 - mse: 0.4852 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - loss: 0.2703 - mse: 0.2703
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2484 - mse: 0.2484
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2437 - mse: 0.2437
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2377 - mse: 0.2377
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2284 - mse: 0.2284
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2318 - mse: 0.2318
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2228 - mse: 0.2228
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2161 - mse: 0.2161
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2145 - mse: 0.2145
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2208 - mse: 0.2208
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2106 - mse: 0.2106
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0596 - mse: 1.0596 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3509 - mse: 0.3509
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2952 - mse: 0.2952
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - loss: 0.2761 - mse: 0.2761
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2604 - mse: 0.2604
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2552 - mse: 0.2552
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2329 - mse: 0.2329
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2383 - mse: 0.2383
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2321 - mse: 0.2321
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - loss: 0.2280 - mse: 0.2280
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2108 - mse: 0.2108
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2257 - mse: 0.2257
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1

- Find the RMSE on train data 

In [23]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_baseline_preds, squared=False)


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.4438064399394116

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [24]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
mean_squared_error(y_train, baseline_preds, squared=False)


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


4042.428499071824

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [27]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
     #Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(50, activation='relu'))


    # THIRD layer
    model.add(layers.Dense(10, activation='relu'))


    # FOURTH layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model
    
    

In [28]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=200, batch_size=256)

In [29]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)

Epoch 1/200


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5368 - mse: 0.5368
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2671 - mse: 0.2671
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2328 - mse: 0.2328
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2151 - mse: 0.2151
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2105 - mse: 0.2105
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2060 - mse: 0.2060
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2010 - mse: 0.2010
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1969 - mse: 0.1969
Epoch 9/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1962 - mse: 0.1962
Epoch 10/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1936 - mse: 0.1936
Epoch 11/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1926 - mse: 0.1926
Epoch 12/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1942 - mse: 0.1942
Epoch 13/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7555 - mse: 0.7555 
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2310 - mse: 0.2310
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2256 - mse: 0.2256
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2146 - mse: 0.2146
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2137 - mse: 0.2137
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2102 - mse: 0.2102
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1995 - mse: 0.1995
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2025 - mse: 0.2025
Epoch 9/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2071 - mse: 0.2071
Epoch 10/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2066 - mse: 0.2066
Epoch 11/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2082 - mse: 0.2082
Epoch 12/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2012 - mse: 0.2012
Epoch 13/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

...

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8661 - mse: 0.8661 
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4548 - mse: 0.4548
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2979 - mse: 0.2979
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2593 - mse: 0.2593
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2552 - mse: 0.2552
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2317 - mse: 0.2317
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2278 - mse: 0.2278
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2160 - mse: 0.2160
Epoch 9/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2263 - mse: 0.2263
Epoch 10/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2141 - mse: 0.2141
Epoch 11/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2129 - mse: 0.2129
Epoch 12/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2054 - mse: 0.2054
Epoch 13/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7611 - mse: 0.7611 
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2162 - mse: 0.2162
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2168 - mse: 0.2168
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2136 - mse: 0.2136
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2128 - mse: 0.2128
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2018 - mse: 0.2018
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2080 - mse: 0.2080
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1979 - mse: 0.1979
Epoch 9/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1987 - mse: 0.1987
Epoch 10/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2041 - mse: 0.2041
Epoch 11/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2070 - mse: 0.2070
Epoch 12/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2027 - mse: 0.2027
Epoch 13/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8675 - mse: 0.8675 
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3036 - mse: 0.3036
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2474 - mse: 0.2474
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2123 - mse: 0.2123
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2194 - mse: 0.2194
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2164 - mse: 0.2164
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1995 - mse: 0.1995
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2024 - mse: 0.2024
Epoch 9/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2052 - mse: 0.2052
Epoch 10/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2008 - mse: 0.2008
Epoch 11/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2017 - mse: 0.2017
Epoch 12/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1976 - mse: 0.1976
Epoch 13/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

In [30]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_bigger_model_preds, squared=False)



c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.47346928236232166

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [45]:
# Define a function that returns a compiled Keras model 
from tensorflow.keras.layers import Dropout
def create_regularized_model():
     #Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_regularizer=regularizers.l2(0.005)))
    model.add(Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(50, activation='relu'))
    model.add(Dropout(0.2))


    # THIRD layer
    model.add(layers.Dense(10, activation='relu'))
 


    # FOURTH layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model
    
    

In [46]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=150, batch_size=256)

In [47]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

Epoch 1/150


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9890 - mse: 0.8198 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4740 - mse: 0.3067
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4533 - mse: 0.2892
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4345 - mse: 0.2734
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4432 - mse: 0.2852
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4094 - mse: 0.2544
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4070 - mse: 0.2550
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4094 - mse: 0.2602
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4012 - mse: 0.2548
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4018 - mse: 0.2581
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3822 - mse: 0.2412
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3738 - mse: 0.2355
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0240 - mse: 0.8495
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7146 - mse: 0.5432
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5097 - mse: 0.3413
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4548 - mse: 0.2894
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4314 - mse: 0.2692
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4272 - mse: 0.2682
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4152 - mse: 0.2592
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4066 - mse: 0.2536
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4049 - mse: 0.2548
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3885 - mse: 0.2413
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4022 - mse: 0.2577
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3952 - mse: 0.2535
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

...

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7677 - mse: 0.5918 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5027 - mse: 0.3299
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4526 - mse: 0.2832
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4579 - mse: 0.2918
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4341 - mse: 0.2712
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4077 - mse: 0.2479
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4164 - mse: 0.2597
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4079 - mse: 0.2542
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4082 - mse: 0.2574
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3945 - mse: 0.2466
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3997 - mse: 0.2546
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3896 - mse: 0.2472
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0942 - mse: 0.9174
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5346 - mse: 0.3605
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4506 - mse: 0.2798
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4447 - mse: 0.2772
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4484 - mse: 0.2842
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4195 - mse: 0.2584
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4255 - mse: 0.2675
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4248 - mse: 0.2698
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4121 - mse: 0.2601
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3986 - mse: 0.2495
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4054 - mse: 0.2593
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3986 - mse: 0.2552
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/st

c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.9009 - mse: 0.7315
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.5034 - mse: 0.3365
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4525 - mse: 0.2889
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4316 - mse: 0.2710
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4176 - mse: 0.2602
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4179 - mse: 0.2635
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4102 - mse: 0.2587
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4019 - mse: 0.2533
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3963 - mse: 0.2505
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3889 - mse: 0.2459
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3820 - mse: 0.2416
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3598 - mse: 0.2221
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━

In [48]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_dropout_preds, squared=False)


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.4395878068440262

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [59]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine

keras_wrapper_3 = KerasRegressor(build_fn=create_regularized_model, epochs=150, batch_size=256)
keras_wrapper_3.fit(X_test_all, y_test_scaled)
y_pred_scaled = keras_wrapper_3.predict(X_test_all)
r2_score_scaled = keras_wrapper_3.score(X_test_all, y_test_scaled)
y_pred = ss_y.inverse_transform(y_pred_scaled)
r2_score = r2_score(y_test, y_pred)

print(f"R^2 Score: {r2_score}")
print(f"R^2 Score (scaled): {r2_score_scaled}")


Epoch 1/150


c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\FLEX 5\anaconda03\envs\learn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1164 - mse: 0.9448 
Epoch 2/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9888 - mse: 0.8187
Epoch 3/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7957 - mse: 0.6269
Epoch 4/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6935 - mse: 0.5261
Epoch 5/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6615 - mse: 0.4958
Epoch 6/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6321 - mse: 0.4681
Epoch 7/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5954 - mse: 0.4331
Epoch 8/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5851 - mse: 0.4245
Epoch 9/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5637 - mse: 0.4048
Epoch 10/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5336 - mse: 0.3764
Epoch 11/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5247 - mse: 0.3693
Epoch 12/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5159 - mse: 0.3622
Epoch 13/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 